In [38]:
import yfinance as yf
import numpy as np
from scipy.stats import norm
from datetime import datetime

def black_scholes(S, K, T, r, sigma, option_type):
    
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 'call':
        option_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        option_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("Option type must be 'call' or 'put'")

    return option_price

def calculate_time_to_expiration(expiration_date):
    today = datetime.now()
    expiration_datetime = datetime.strptime(expiration_date, '%Y-%m-%d')
    T = (expiration_datetime - today).days / 252  # Divide by 252 to express in years
    return T

stock_option = yf.Ticker("MRNA")

expiration_date = '2024-03-15'

while True:
    # We get the call and put option data for the specified expiration date
    option_chain = stock_option.option_chain(expiration_date)

    # Ask the user to input the strike price (K)
    K = float(input("Enter the strike price (K): "))

    # Extract call and put prices for the specified strike price (K)
    current_call_price = option_chain.calls[option_chain.calls['strike'] == K]['lastPrice'].iloc[0] if not option_chain.calls.empty else np.nan
    current_put_price = option_chain.puts[option_chain.puts['strike'] == K]['lastPrice'].iloc[0] if not option_chain.puts.empty else np.nan

    # Calculate remaining parameters and print results
    stock_data = yf.Ticker("MRNA").history(start="2000-01-01")
    
    S = stock_data['Close'].iloc[-1]  # Current stock price
    daily_returns = stock_data['Close'].pct_change().dropna()
    sigma = daily_returns.ewm(span=252).std().iloc[-1]  # Assuming 252 trading days in a year
    treasury_yield = yf.Ticker("MRNA").history(period="1d").Close.iloc[-1] / 100  # Convert percentage to decimal
    r = treasury_yield
    T = calculate_time_to_expiration(expiration_date)

    call_price = black_scholes(S, K, T, r, sigma, 'call')
    put_price = black_scholes(S, K, T, r, sigma, 'put')

    print(f"Call Option Price for Strike Price {K}: {current_call_price:.2f}")
    print(f"Calculated Call Option Price for Strike Price {K}: {call_price:.2f}")
    print(f"Put Option Price for Strike Price {K}: {current_put_price:.2f}")
    print(f"Calculated Put Option Price for Strike Price {K}: {put_price:.2f}")

    call_difference = (1 - (call_price/current_call_price))*100
    put_difference = (1 - (put_price/current_put_price))*100
    print(f"Current Stock Price: {S:.2f}")
    print(f"Strike Price: {K:.2f}")

    # Ask user if they want to continue
    cont = input("Do you want to enter another strike price? (y/n): ")
    if cont.lower() != 'y':
        break


Enter the strike price (K):  70


Call Option Price for Strike Price 70.0: 16.30
Calculated Call Option Price for Strike Price 70.0: 22.36
Put Option Price for Strike Price 70.0: 1.16
Calculated Put Option Price for Strike Price 70.0: 0.00
Current Stock Price: 85.78
Strike Price: 70.00


Do you want to enter another strike price? (y/n):  y
Enter the strike price (K):  80


Call Option Price for Strike Price 80.0: 9.40
Calculated Call Option Price for Strike Price 80.0: 13.30
Put Option Price for Strike Price 80.0: 3.57
Calculated Put Option Price for Strike Price 80.0: 0.00
Current Stock Price: 85.78
Strike Price: 80.00


Do you want to enter another strike price? (y/n):  y
Enter the strike price (K):  85


...

Do you want to enter another strike price? (y/n):  y
Enter the strike price (K):  90


Call Option Price for Strike Price 90.0: 4.30
Calculated Call Option Price for Strike Price 90.0: 4.24
Put Option Price for Strike Price 90.0: 8.85
Calculated Put Option Price for Strike Price 90.0: 0.00
Current Stock Price: 85.78
Strike Price: 90.00


Do you want to enter another strike price? (y/n):  y
Enter the strike price (K):  100


Call Option Price for Strike Price 100.0: 1.75
Calculated Call Option Price for Strike Price 100.0: 0.00
Put Option Price for Strike Price 100.0: 16.05
Calculated Put Option Price for Strike Price 100.0: 4.82
Current Stock Price: 85.78
Strike Price: 100.00


Do you want to enter another strike price? (y/n):  n
